# Using GeoPandas

**Setting up the conda env:**

```
conda create -n geo python=3.8
conda activate geo
conda install mamba -c conda-forge
mamba install geemap geopandas descartes rtree=0.9.3 -c conda-forge
mamba install ipython-sql sqlalchemy psycopg2 -c conda-forge
```

**Sample dataset:**
- [nyc_data.zip](https://github.com/giswqs/postgis/raw/master/data/nyc_data.zip) (Watch this [video](https://youtu.be/fROzLrjNDrs) to load data into PostGIS)

**References**:
- [Introduction to PostGIS](https://postgis.net/workshops/postgis-intro)
- [Using SQL with Geodatabases](https://desktop.arcgis.com/en/arcmap/latest/manage-data/using-sql-with-gdbs/sql-and-enterprise-geodatabases.htm)

## Connecting to the database

In [ ]:
import os
from sqlalchemy import create_engine

In [ ]:
host = "localhost"
database = "nyc"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

In [ ]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [ ]:
engine = create_engine(connection_string)

In [ ]:
from sqlalchemy import inspect

In [ ]:
insp = inspect(engine)
insp.get_table_names()

## Reading data from PostGIS

In [ ]:
import geopandas as gpd

In [ ]:
sql = 'SELECT * FROM nyc_neighborhoods'

In [ ]:
gdf = gpd.read_postgis(sql, con=engine)

In [ ]:
gdf

In [ ]:
gdf.crs

## Writing files

In [ ]:
out_dir = os.path.expanduser('~/Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
out_json = os.path.join(out_dir, 'nyc_neighborhoods.geojson')
gdf.to_file(out_json, driver="GeoJSON")

In [ ]:
out_shp = os.path.join(out_dir, 'nyc_neighborhoods.shp')
gdf.to_file(out_shp)

In [ ]:
gdf.crs

## Measuring area

In [ ]:
gdf = gdf.set_index("name")

In [ ]:
gdf["area"] = gdf.area
gdf["area"]

## Getting polygon bounary

In [ ]:
gdf['boundary'] = gdf.boundary
gdf['boundary']

## Getting polygon centroid

In [ ]:
gdf['centroid'] = gdf.centroid
gdf['centroid']

## Making maps

In [ ]:
gdf.plot()

In [ ]:
gdf.plot("area", legend=True, figsize=(10, 8))

In [ ]:
gdf = gdf.set_geometry("centroid")
gdf.plot("area", legend=True,figsize=(10, 8))

In [ ]:
ax = gdf["geom"].plot(figsize=(10, 8))
gdf["centroid"].plot(ax=ax, color="black")

In [ ]:
gdf = gdf.set_geometry("geom")

## Reprojecting data

In [ ]:
sql = 'SELECT * FROM nyc_neighborhoods'

In [ ]:
gdf = gpd.read_postgis(sql, con=engine)

In [ ]:
gdf_crs = gdf.to_crs(epsg="4326")

In [ ]:
gdf_crs

In [ ]:
geojson = gdf_crs.__geo_interface__

## Displaying data on an interative map

In [ ]:
import geemap

In [ ]:
m = geemap.Map(center=[40.7341, -73.9113], zoom=10, ee_initialize=False)
m

In [ ]:
style = {
    "stroke": True,
    "color": "#000000",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "#0000ff",
    "fillOpacity": 0.4,
}

In [ ]:
m.add_geojson(geojson, style=style, layer_name="nyc neighborhoods")

In [ ]:
sql2 = 'SELECT * FROM nyc_subway_stations'

In [ ]:
gdf_subway = gpd.read_postgis(sql2, con=engine)

In [ ]:
gdf_subway_crs = gdf_subway.to_crs(epsg="4326")

In [ ]:
subway_geojson = gdf_subway_crs.__geo_interface__

In [ ]:
m.add_geojson(subway_geojson, layer_name="nyc subway stations")

In [ ]:
sql3 = "SELECT * FROM nyc_census_blocks WHERE boroname='Manhattan'"

In [ ]:
gdf_blocks = gpd.read_postgis(sql3, con=engine)

In [ ]:
gdf_blocks_crs = gdf_blocks.to_crs(epsg="4326")

In [ ]:
blocks_geojson = gdf_blocks_crs.__geo_interface__

In [ ]:
m.add_geojson(blocks_geojson, style=style, layer_name="nyc census blocks")

In [ ]:
sql4 = "SELECT geom FROM nyc_homicides WHERE boroname='Manhattan'"

In [ ]:
gdf_homicides = gpd.read_postgis(sql4, con=engine)

In [ ]:
gdf_homicides_crs = gdf_homicides.to_crs(epsg="4326")

In [ ]:
homicides_geojson =gdf_homicides_crs.__geo_interface__

In [ ]:
m.add_geojson(homicides_geojson, style=style, layer_name="nyc homicides")